In [78]:
import os
import pandas as pd
import numpy as np
import cv2

import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as T

from torch.utils.data import Dataset, DataLoader

from understanding_clouds.utils import preproces_dataframe_all_masks, get_all_masks_and_img
from understanding_clouds.constants import LABELS_MAPPING

import os
import argparse
import json

from understanding_clouds.torchvision_references.engine import train_one_epoch, evaluate
from understanding_clouds.torchvision_references.utils import collate_fn

In [79]:
class MaskUnetDataset(Dataset):
    def __init__(self, images_dirpath, transforms=None, img_scale_factor=4, subsample=None):
        self.images_dirpath = images_dirpath
        self._img_scale_factor = 4
        df = pd.read_csv(os.path.join(images_dirpath, 'train.csv'))
        df = preproces_dataframe_all_masks(df)
        self.df = df.iloc[::subsample] if subsample is not None else df
        self.transforms = transforms

    def __getitem__ (self, index):
        masks, img, labels = get_all_masks_and_img(
            self.df, index, os.path.join(self.images_dirpath, 'train_images'), scale_factor=self._img_scale_factor)
        
        img = cv2.resize(img, (352,528), interpolation=cv2.INTER_AREA)
        masks = [cv2.resize(mask, (352,528), interpolation=cv2.INTER_AREA) for mask in masks]
        
        labels = [LABELS_MAPPING[l] for l in labels]
        labels = [l for l in labels if l > 0]
        masks_not_empty = []
        #for mask in filter(lambda x: np.sum(x) > 0, masks):
        #    masks_not_empty.append(mask)
        
        img_id = torch.tensor([index])
        labels = torch.as_tensor(labels, dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8) / 255
        iscrowd = torch.zeros((len(masks),), dtype=torch.int64)
        
        img = T.ToTensor()(img)
        if self.transforms is not None:
            img, target = self.transforms(img, target)

        img = img.to('cuda')
        masks = masks.to('cuda').float()
        
        return img, masks
        
    def __len__(self):
        return len(self.df)

In [80]:
torch.cuda.is_available()

True

In [81]:
device = torch.device(
            'cuda' if torch.cuda.is_available() else 'cpu')

model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=4, init_features=32, pretrained=False).to(device)

#model = model.to(device)

Using cache found in /home/jfklama/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


In [82]:
class DiceLoss(nn.Module):
    __name__ = 'dice_loss'

    def __init__(self, eps=1e-7, activation='sigmoid'):
        super().__init__()
        self.activation = activation
        self.eps = eps

    def forward(self, y_pr, y_gt):
        return 1 - f_score(y_pr, y_gt, beta=1., 
                           eps=self.eps, threshold=None, 
                           activation=self.activation)

NameError: name 'nn' is not defined

In [83]:
train_dataset = MaskUnetDataset(images_dirpath = '../data/', subsample = 500)
#dataloaders = {'TRAIN': torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)}
#dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)
dataloader = DataLoader( train_dataset, batch_size=1, shuffle=True )
loss_fn = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
gamma = 0.9

lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(
            optimizer=optimizer, gamma=gamma)

experiment_dirpath = '.'

In [84]:
def save_model(epoch: int):
        os.makedirs(experiment_dirpath, exist_ok=True)
        checkpoint = {'epoch': epoch,
                      'state_dict': model.state_dict(),
                      'optimizer': optimizer.state_dict(),
                      'lr_scheduler': lr_scheduler.state_dict()}
        torch.save(checkpoint, os.path.join(
            experiment_dirpath, 'model.pth'))

In [85]:
def _single_forward_pass(images, targets, phase):
        images = [img.cuda() for img in images]
        targets = [{k: v.cuda() for k, v in target.items()}
                   for target in targets]
        # it has a structure of {'loss_type': torch.tensor corresponding to the loss}
        loss_dict = model(images)
        return loss_dict

In [86]:
'''
def train(dataloaders = dataloaders, epochs: int = 10, snapshot_frequency: int = 10):
        for i, epoch in enumerate(range(1, epochs + 1)):
            for phase, dataloader in dataloaders.items():
                if phase == 'TRAIN':
                    train_one_epoch(model, optimizer, dataloader, device, epoch, print_freq=len(dataloader) // 5)
                    lr_scheduler.step()
                else:
                    evaluate(model, dataloader, device=device)
            if i % snapshot_frequency == 0:
                save_model(epoch)
        save_model(epoch)
'''

def train(dataloader = dataloader, epochs: int = 10, snapshot_frequency: int = 10):
    
    loss_list = []
    
    for i, epoch in enumerate(range(1, epochs + 1)):
        print(f'Epoch {epoch}')
        model.train()
        
        loss_tmp = []
        
        for image, masks in dataloader:
            #print(image[0].shape)
            input_batch = image[0].unsqueeze(0).type(torch.cuda.FloatTensor)
            input_batch = input_batch.to('cuda')
            #image = image.to('cuda')
            #masks = masks[0].unsqueeze(0).type(torch.cuda.FloatTensor)
            #masks = masks.to('cuda').float()
            
            optimizer.zero_grad()
            #output = model(input_batch)
            output = model(image)
            loss = loss_fn(output, masks)
            loss.backward()
            optimizer.step()
        '''    
        for i in range(len(train_dataset)):
                    image, mask = train_dataset[i]
                    #image = image.cuda
                    #mask = mask.cuda()
                    input_batch = image.unsqueeze(0).type(torch.cuda.FloatTensor)
                    input_batch = input_batch.to('cuda')
                    mask = mask.to('cuda').float()
                    
                    mask = mask.unsqueeze(0).type(torch.cuda.FloatTensor)
                    #print(image.size(), mask.size(), input_batch.size())
                    #mask = mask.float()
                    #print(type(input_batch))
                    
                    
                    optimizer.zero_grad()
                    #print(images.shape)
                    #with torch.set_grad_enabled(phase == 'TRAIN'):
                    #    loss_dict = _single_forward_pass(images, targets, phase)
                    #    loss = sum([l for l in loss_dict.values()])
                    #    phase_loss += loss.item()
                    output = model(input_batch)
                    loss = loss_fn(output, mask)
                    #print(loss.item())
                    loss_tmp.append(loss.item())
                    
                    #if phase == 'TRAIN':
                    loss.backward()
                    optimizer.step()'''
        loss_list.append( np.mean(loss_tmp) )
        lr_scheduler.step()
        if i % snapshot_frequency == 0:
                save_model(epoch)
    print(loss_list) 
    save_model(epoch)

In [87]:
train()

Epoch 1
Epoch 2


KeyboardInterrupt: 